In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
from scipy import linalg as la
sp.init_printing()
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import torch
import torch.nn as nn

from sklearn import datasets
import seaborn as sns

In [14]:
from google.colab import files
uploaded = files.upload()
import io
df = pd.read_csv(io.BytesIO(uploaded['Walmart (13).csv']))
print(df.head())

Saving Walmart.csv to Walmart (13).csv
   Store        Date  Weekly_Sales  Holiday_Flag  Temperature  Fuel_Price  \
0      1  05-02-2010    1643690.90             0        42.31       2.572   
1      1  12-02-2010    1641957.44             1        38.51       2.548   
2      1  19-02-2010    1611968.17             0        39.93       2.514   
3      1  26-02-2010    1409727.59             0        46.63       2.561   
4      1  05-03-2010    1554806.68             0        46.50       2.625   

          CPI  Unemployment  
0  211.096358         8.106  
1  211.242170         8.106  
2  211.289143         8.106  
3  211.319643         8.106  
4  211.350143         8.106  


CPI = how expensive goods/services are on average.
It’s used to understand inflation’s impact on things like sales, wages, or spending patterns.
A higher CPI means the cost of living was higher (prices increased)
A lower CPI means prices were lower or inflation was lower.

In [15]:
df=pd.read_csv('Walmart (13).csv')
df.shape

(6435, 8)

In [16]:
df

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
0,1,05-02-2010,1643690.90,0,42.31,2.572,211.096358,8.106
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
2,1,19-02-2010,1611968.17,0,39.93,2.514,211.289143,8.106
3,1,26-02-2010,1409727.59,0,46.63,2.561,211.319643,8.106
4,1,05-03-2010,1554806.68,0,46.50,2.625,211.350143,8.106
...,...,...,...,...,...,...,...,...
6430,45,28-09-2012,713173.95,0,64.88,3.997,192.013558,8.684
6431,45,05-10-2012,733455.07,0,64.89,3.985,192.170412,8.667
6432,45,12-10-2012,734464.36,0,54.47,4.000,192.327265,8.667
6433,45,19-10-2012,718125.53,0,56.47,3.969,192.330854,8.667


In [18]:
df.dtypes

,0
Store,int64
Date,object
Weekly_Sales,float64
Holiday_Flag,int64
Temperature,float64
Fuel_Price,float64
CPI,float64
Unemployment,float64


In [29]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Store,6435.0,2.300000e+01,12.988182,1.000,12.000,23.000000,3.400000e+01,4.500000e+01
Weekly_Sales,6435.0,1.046965e+06,564366.622054,209986.250,553350.105,960746.040000,1.420159e+06,3.818686e+06
Holiday_Flag,6435.0,6.993007e-02,0.255049,0.000,0.000,0.000000,0.000000e+00,1.000000e+00
Temperature,6435.0,6.066378e+01,18.444933,-2.060,47.460,62.670000,7.494000e+01,1.001400e+02
Fuel_Price,6435.0,3.358607e+00,0.459020,2.472,2.933,3.445000,3.735000e+00,4.468000e+00
CPI,6435.0,1.715784e+02,39.356712,126.064,131.735,182.616521,2.127433e+02,2.272328e+02
Unemployment,6435.0,7.999151e+00,1.875885,3.879,6.891,7.874000,8.622000e+00,1.431300e+01


In [20]:
df['Weekly_Sales'].value_counts()

,count
Weekly_Sales,
760281.43,1
1643690.90,1
1641957.44,1
843361.10,1
773367.71,1
...,...
1404429.92,1
1472515.79,1
1439541.59,1


In [21]:
df['CPI'].value_counts()

,count
CPI,
126.526286,11
130.071032,11
130.737871,11
130.719581,11
130.701290,11
...,...
210.337426,1
210.339968,1
210.389546,1


In [23]:
df.loc[df.Weekly_Sales.isna()]  # Looking for missing values/part

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment


In [25]:
df.isna().sum() # making sure there is no missing part by trying to locate it

,0
Store,0
Date,0
Weekly_Sales,0
Holiday_Flag,0
Temperature,0
Fuel_Price,0
CPI,0
Unemployment,0


In [27]:
df.loc[df.Holiday_Flag==1]

,Store,Date,Weekly_Sales,Holiday_Flag,Temperature,Fuel_Price,CPI,Unemployment
1,1,12-02-2010,1641957.44,1,38.51,2.548,211.242170,8.106
31,1,10-09-2010,1507460.69,1,78.69,2.565,211.495190,7.787
42,1,26-11-2010,1955624.11,1,64.52,2.735,211.748433,7.838
47,1,31-12-2010,1367320.01,1,48.43,2.943,211.404932,7.838
53,1,11-02-2011,1649614.93,1,36.39,3.022,212.936705,7.742
...,...,...,...,...,...,...,...,...
6375,45,09-09-2011,746129.56,1,71.48,3.738,186.673738,8.625
6386,45,25-11-2011,1170672.94,1,48.71,3.492,188.350400,8.523
6391,45,30-12-2011,869403.63,1,37.79,3.389,189.062016,8.523
6397,45,10-02-2012,803657.12,1,37.00,3.640,189.707605,8.424


In [36]:
df.loc[(df.Fuel_Price<4.472)& (df.Holiday_Flag==1),['Store', 'Date', 'Weekly_Sales', 'Fuel_Price', 'Holiday_Flag'] ]

,Store,Date,Weekly_Sales,Fuel_Price,Holiday_Flag
1,1,12-02-2010,1641957.44,2.548,1
31,1,10-09-2010,1507460.69,2.565,1
42,1,26-11-2010,1955624.11,2.735,1
47,1,31-12-2010,1367320.01,2.943,1
53,1,11-02-2011,1649614.93,3.022,1
...,...,...,...,...,...
6375,45,09-09-2011,746129.56,3.738,1
6386,45,25-11-2011,1170672.94,3.492,1
6391,45,30-12-2011,869403.63,3.389,1
6397,45,10-02-2012,803657.12,3.640,1
